<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
#Quick import of our most commonly used libraries. In the case of seaborn, I just happen to like it.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel

In [97]:
#Loading the data and having a quick look, immediately confirming the warned missing values
voting_df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data')
#voting_df.head()
#Let's fix that ? column here as well, it's easy to see it's mean't to be n.5
#I'll also go ahead and rename the republican column to rep/dem to be more descriptive
voting_df.rename(columns = {'republican':'rep/dem','?':'n.5'}, inplace = True)
#One more quick look to insure that came out right.
voting_df.head()

,rep/dem,n,y,n.1,y.1,y.2,y.3,n.2,n.3,n.4,y.4,n.5,y.5,y.6,y.7,n.5,y.8
0,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
1,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
2,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
3,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
4,democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y


In [98]:
#We were lucky that the first five lines were so telling of missing data
#However, this is a good example of missing data that might hide from us
voting_df.isna().sum()

rep/dem    0
n          0
y          0
n.1        0
y.1        0
y.2        0
y.3        0
n.2        0
n.3        0
n.4        0
y.4        0
n.5        0
y.5        0
y.6        0
y.7        0
n.5        0
y.8        0
dtype: int64

In [99]:
#Looking closely, we can see that there are ?'s in our dataframe, at least somewhere
#So next, we'll try and handle those. First, I'm going to get those letters formatted
voting_df.replace(('y', 'n'), (0,1), inplace=True)
voting_df.head()

,rep/dem,n,y,n.1,y.1,y.2,y.3,n.2,n.3,n.4,y.4,n.5,y.5,y.6,y.7,n.5,y.8
0,republican,1,0,1,0,0,0,1,1,1,1,1,0,0,0,1,?
1,democrat,?,0,0,?,0,0,1,1,1,1,0,1,0,0,1,1
2,democrat,1,0,0,1,?,0,1,1,1,1,0,1,0,1,1,0
3,democrat,0,0,0,1,0,0,1,1,1,1,0,?,0,0,0,0
4,democrat,1,0,0,1,0,0,1,1,1,1,1,1,0,0,0,0


In [100]:
#First I'm converting the ?'s into actual nan values to make them easier to access
voting_df.replace('?',np.NaN, inplace = True)
voting_df.head()

,rep/dem,n,y,n.1,y.1,y.2,y.3,n.2,n.3,n.4,y.4,n.5,y.5,y.6,y.7,n.5,y.8
0,republican,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN
1,democrat,NaN,0.0,0.0,NaN,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
2,democrat,1.0,0.0,0.0,1.0,NaN,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
3,democrat,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,NaN,0.0,0.0,0.0,0.0
4,democrat,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0


In [85]:
#I went ahead and did a straight forwards forward, then backward fill to make sure
#all values were covered. 
voting_df.fillna(method = 'ffill', inplace = True)
voting_df.fillna(method = 'bfill', inplace = True)
voting_df.head()


,rep/dem,n,y,n.1,y.1,y.2,y.3,n.2,n.3,n.4,y.4,n.5,y.5,y.6,y.7,n.5,y.8
0,republican,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1,0.0,0.0,0.0,1,1.0
1,democrat,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0,1.0,0.0,0.0,1,1.0
2,democrat,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0,1.0,0.0,1.0,1,0.0
3,democrat,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0,1.0,0.0,0.0,0,0.0
4,democrat,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1,1.0,0.0,0.0,0,0.0


In [92]:
#I struggled a lot trying to figure out a way to make a function that would
#Iterate over the columns and handle this one set at a time but just couldn't figure it out
#However, this does tell us that the very first issue, doesn't have much difference either way
stat, pvalue = ttest_ind(voting_df['n'],voting_df['y'])
print(stat, pvalue)

1.2928622833259626 0.19640350132878914
